## Restricted Boltzmann Machine (RBM)
A [RBM](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine) is a generative stochastic artificial neural network that can learn a probability distribution over its set of inputs. 

A standard RBM has following diagram
<img src="../assets/rbm-diagram.png" alt="rbm-diagram" style="width: 40%; height: 40%">

where we denote
* $v\in\mathbb{R}^D$ is visible units
* $h\in\mathbb{R}^H$ is hidden units

And $v,h$ takes binary value (0,1), then it defines an energy function (similar as Hopfield network)
$$
E(v,h) = -a^Tv -b^Th - v^TWh
$$
which allows us to model the joint-distribution $(v,h)$ in term of the energy function i.e
$$
P(v,h) = \frac{1}{Z}e^{-E(v,h)}
$$
where $Z$ is normalized constant i.e
$$
Z = \sum_{v,h}e^{-E(v,h)}
$$

The above diagram is a bipartite graph which allows to define conditional probability 
$$
\begin{array}{rl}
P(v|h) &= \prod_{i=1}^DP(v_i|h)\\
P(h|v) &= \prod_{j=1}^HP(h_j|v)
\end{array}
$$
where individual probability is given by
$$
\begin{array}{rl}
P(v_i=1|h) &= \sigma\left(a_i + \sum_{j=1}^{H}w_{i,j}h_j\right)\\
P(h_j=1|v) &= \sigma\left(b_j + \sum_{i=1}^Dw_{i,j}v_i\right)
\end{array}
$$

In this note, we look at the following task
* given a set of unlabelled dataset $\left\{v^{(i)}\right\}_{i=1}^n$
* given a configuration of RMB e.g number of hidden units $H$

The goal is to learn $a$, $b$ and $W$ that model the joint-distribution directly from dataset $\left\{v^{(i)}\right\}_{i=1}^n$ i.e that maximize the sum of log-likelihood
$$
\sum_{i=1}^n\log\left(P(v^{(i)})\right)
$$
where 
$$
P(v) = \sum_{h}P(v,h)=\frac{1}{Z}\sum_{h}e^{-E(v,h)}
$$
The learning for RBM is difficult since computation of $Z$ is exponentially hard, however we have the following interesting fact
$$
\frac{\partial \log P(v)}{\partial v} = <v_i,h_j>_{data} - <v_i,h_j>_{model}
$$
where
* $<v_i, h_j>_{data}$ is expected value of the product when $v$ is clamped on data (i.e $v_i$ is taken from training dataset)
* $<v_i, h_j>_{model}$ is expected value of the product when $v,h$ is generated by model

Now let's look at how we compute $<v_i,h_j>_{data}$ and $<v_i,h_j>_{model}$, it's done via two phases
* **positive phase**: clamp a datavector on the visible units, then we sample $h$ and compute the average $<v,h>$
* **negative phase**: keep a set of fantasy particles $v$ where each particle has a value that is a global configuration, then we sample $h$ and compute the average $<v,h>$

To illustrate the process, we show the following diagram is taken from G. Hinton's course (NN)
<img src="../assets/rbm-pos-neg.png" alt="rbm-pos-neg" style="width: 60%; height: 50%">

and the update rule for $W$
$$
\Delta w_{i,j} = \epsilon\left( <v_i,h_j>^0 - <v_i,h_j>^\infty \right)
$$
This is actually based on Markov Chain Monte Carlo (MCMC) and Gibbs sampling. However this's still not efficient enough. 

Fortunately, G. Hinton introduces a very fast learning algorithm so called **Constrative Divergence Learning** where we can use a short cut
<img src="../assets/rbm-cd-learning.png" alt="rbm-cd-learning" style="width: 60%; height: 50%">
we denote 
$$
CD_1 = <v_i,h_j>^0 - <v_i,h_j>^1
$$
we can similarly define $CD_t = <v_i,h_j>^0 - <v_i,h_j>^t$. One can see that the learning doesn't use the correct gradient of the log-likelihood, as suggested by G. Hinton, we can use the following process
* start with small weight $w_{i,j}$ and we use $CD_1$
* once the weights grow (after some number of iterations), we can use $CD_3$, then $CD_{10}$

We will consider MNIST dataset for this task, then later we go through some RBM's application.